In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
import glob
import gensim 
from gensim.models import Word2Vec 
from gensim.test.utils import common_texts
from sklearn.metrics import classification_report

Using TensorFlow backend.
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [2]:
def get_latest_file_name(path, ext=''):
    list_of_files = glob.glob(f'{path}/*{ext}')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    return latest_file

In [3]:
# Load the latest model
model = get_latest_file_name('saved_model/')
print(f"Using {model}...")

model = load_model(model)

Using saved_model/MODEL_cbow_tokens_26-03-2020_13:13:35...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [4]:
# model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 500, 100)     12123100    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 499, 200)     40200       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 498, 200)     60200       embedding_1[0][0]                
____________________________________________________________________________________________

In [5]:
# Load the latest test data
test_data = get_latest_file_name('saved_test_data/', '.npy')
print(f"Using test data:`{test_data}`...")

test_data = np.load(test_data)

Using test data:`saved_test_data/test_data_26-03-2020_13:13:35.npy`...


In [6]:
# Load the ground truth test data
gt_test = pd.read_csv("cleaned_data/cleaned_test_data.csv")

In [7]:
predictions = model.predict(test_data, batch_size=1024, verbose=1)

25000/25000 [==============================] - 247s 10ms/step


In [8]:
# Model.predict returns an array with two columns. 
# Column one represents the probability for the sample to be label 1.
# Column 2 represents probability for the sample to be label 2.
# So, we swap the columns of the predictions
predictions[:,[0, 1]] = predictions[:,[1, 0]]

In [9]:
prediction_labels = predictions.argmax(axis=1)
acc = sum(gt_test.label==prediction_labels)/len(prediction_labels)
print(classification_report(gt_test.label, prediction_labels, target_names=["Negative class (0)","Positive class (1)"]))

                    precision    recall  f1-score   support

Negative class (0)       0.87      0.91      0.89     12500
Positive class (1)       0.91      0.86      0.88     12500

          accuracy                           0.89     25000
         macro avg       0.89      0.89      0.89     25000
      weighted avg       0.89      0.89      0.89     25000

